In [ ]:
from BERT_model import *
import BERT_model
import pandas as pd
import numpy as np
import torch
from torch import nn

In [ ]:
df = pd.read_csv('../data/csv/MBTI_1000.csv')

In [ ]:
df_train, df_val, df_test = np.split(df.sample(frac=1), [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

In [ ]:
EPOCHS = 12
model = BertClassifier()
LR = 1e-6

In [ ]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    device, criterion, optimizer = BERT_model.Get_Device_Optimizer(model, learning_rate)
    
    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):
                output = BERT_model.GetModelOutput(model, train_input, train_label, device)
                total_loss_train, batch_loss = BERT_model.Get_Total_Loss(output, criterion, train_label)                
                total_acc_train = BERT_model.Get_Total_Accuracy(output, train_label)                

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:
                    output = BERT_model.GetModelOutput(model, val_input, val_label, device)
                    total_loss_val, batch_loss = BERT_model.Get_Total_Loss(output, criterion, val_label)                    
                    total_acc_val = BERT_model.Get_Total_Accuracy(output, val_label)

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
              
train(model, df_train, df_val, LR, EPOCHS)

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    device = BERT_model.Get_Device(model)

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:
            output = BERT_model.GetModelOutput(model, test_input, test_label, device)
            total_acc_test = BERT_model.Get_Total_Accuracy(output, test_label)            
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)